In [1]:
# coding: utf-8
"""
UMで精度の検証
"""
from library.filer import Filer
from library.sampleUM import UM
from library.dnp import Evaluation
import glob
import numpy as np

In [2]:
list_filepath = glob.glob('files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/*.txt')
list_testfile = Filer.readdump('./files/rakuten_corpus/rakuten_corpus_master/testfile/list_sepword_label.dump')
list_testword = [row[0] for row in list_testfile]
list_label = [row[1] for row in list_testfile]

In [ ]:
for path in list_filepath:
    print path
    path_rev = path.replace('files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/', '')
    Filer.writetxt([path_rev], './files/result/rakuten/experiment1/UM.txt')
    for counter in range(10):
        list_word = Filer.readtxt(path)
        list_word = [row.split(' ') for row in list_word]
        um = UM(alpha=1.0, beta=1.0, K=6, converge=0.01, max_iter=100)
        um.set_param(list_d_words=list_word)
        um.fit()
    
        list_predict = []
        for row in list_testword:
            try:
                list_prob = um.infer(row)
                list_predict.append(np.argmax(list_prob))
            except KeyError:
                list_predict.append(None)
        
        list_predict_rev = []
        list_label_rev = []
        for predict, label in zip(list_predict, list_label):
            if predict != None:
                list_predict_rev.append(predict)
                list_label_rev.append(label)

        eva = Evaluation()
        dict_result = eva.cal_f_measure(list_predict_rev, list_label_rev)
        Filer.writetxt([' '.join([str(dict_result['purity']),str(dict_result['invpurity']),str(dict_result['fvalue'])])],
                       './files/result/rakuten/experiment1/UM.txt')

In [ ]:
list_theta = um.list_theta
list_dict_phi = um.list_dict_phi

In [ ]:
n = 1
for row in sorted(um.list_dict_phi[n].items(), key=lambda x: x[1], reverse=True)[0:20]:
    print row[0], row[1]